In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import os
from six.moves.urllib.request import urlopen
import numpy as np
import pandas as pd

from keras import backend as K

#Clear TF memory
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

C:\Users\StrikeWade\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Dataset download

In [2]:
#IRIS Datasets
IRIS_training = 'iris_training.csv'
IRIS_training_url = 'http://download.tensorflow.org/data/iris_training.csv'

IRIS_test = "iris_test.csv"
IRIS_test_url = "http://download.tensorflow.org/data/iris_test.csv"

In [ ]:
#If dataset not exist, download
def download():
    if not os.path.exists(IRIS_training):
        train_raw = urlopen(IRIS_training_url).read()
        with open(IRIS_training, 'wb') as f:
            f.write(train_raw)
            
    if not os.path.exists(IRIS_test):
        test_raw = urlopen(IRIS_test_url).read()
        with open(IRIS_test, 'wb') as f:
            f.write(test_raw)
            
download()

# Load downloaded CSV files as TF datasets

In [3]:
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(filename=IRIS_training, 
                                                                   target_dtype=np.int, 
                                                                   features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(filename=IRIS_test, 
                                                               target_dtype=np.int, 
                                                               features_dtype=np.float32)

In [4]:
training_set

Dataset(data=array([[6.4, 2.8, 5.6, 2.2],
       [5. , 2.3, 3.3, 1. ],
       [4.9, 2.5, 4.5, 1.7],
       [4.9, 3.1, 1.5, 0.1],
       [5.7, 3.8, 1.7, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [6.9, 3.1, 5.1, 2.3],
       [6.7, 3.1, 4.4, 1.4],
       [5.1, 3.7, 1.5, 0.4],
       [5.2, 2.7, 3.9, 1.4],
       [6.9, 3.1, 4.9, 1.5],
       [5.8, 4. , 1.2, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [7.7, 3.8, 6.7, 2.2],
       [6.3, 3.3, 4.7, 1.6],
       [6.8, 3.2, 5.9, 2.3],
       [7.6, 3. , 6.6, 2.1],
       [6.4, 3.2, 5.3, 2.3],
       [5.7, 4.4, 1.5, 0.4],
       [6.7, 3.3, 5.7, 2.1],
       [6.4, 2.8, 5.6, 2.1],
       [5.4, 3.9, 1.3, 0.4],
       [6.1, 2.6, 5.6, 1.4],
       [7.2, 3. , 5.8, 1.6],
       [5.2, 3.5, 1.5, 0.2],
       [5.8, 2.6, 4. , 1.2],
       [5.9, 3. , 5.1, 1.8],
       [5.4, 3. , 4.5, 1.5],
       [6.7, 3. , 5. , 1.7],
       [6.3, 2.3, 4.4, 1.3],
       [5.1, 2.5, 3. , 1.1],
       [6.4, 3.2, 4.5, 1.5],
       [6.8, 3. , 5.5, 2.1],
 

In [5]:
#Specify that all features have real-value data
feature_columns = [tf.feature_column.numeric_column(key='x', shape=[4])]
feature_columns

[_NumericColumn(key='x', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

# Define input functions

In [6]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x': np.array(training_set.data)}, 
                                                    y=np.array(training_set.target), 
                                                    num_epochs=None, 
                                                    shuffle=True)
test_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x': np.array(test_set.data)}, 
                                                   y=np.array(test_set.target), 
                                                   num_epochs=1, 
                                                   shuffle=False)

# Hyperparameter study with various activation functions
## Hyperparameter list

In [7]:
import time
learning_rate = [0.0001 * 2 ** (x) for x in range(1,12,2)]
dropout_prob = [0.01, 0.05, 0.1, 0.25]
l1_reg = [0.01, 0.1, 1.0, 10.0]
train_steps = [100, 500, 2500]

## Sigmoid activation

In [ ]:
#Loop counts to keep track of whole training run over the multi-dimensional hyperparams space
loops = len(learning_rate) * len(dropout_prob) * len(train_steps) * len(l1_reg)
loop_remain = loops

#Empty lists to append to
accuracy_sigmoid_temp = []
time_sigmoid_temp = []

## Main training loop to evaluate sigmoid activation

In [ ]:
#Remove all the unecessary warning
from tensorflow import logging
logging.set_verbosity(logging.ERROR)

for d in dropout_prob:
    for l in learning_rate:
        for r in l1_reg:
            for t in train_steps:
                
                #Start time
                start = time.time()
                
                #Main DNN classifier to train
                classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns, 
                                                        hidden_units=[5,10,5], 
                                                        dropout=d, 
                                                        n_classes=3, 
                                                        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=l, 
                                                                                                    l1_regularization_strength=r), 
                                                        activation_fn=tf.nn.sigmoid)
                
                classifier.train(input_fn=train_input_fn, steps=t)
                
                accuracy_sigmoid_temp.append(classifier.evaluate(input_fn=test_input_fn)['accuracy'])
                
                end = time.time()
                
                loop_remain -= 1
                
                percentage_done = round(100*(loops-loop_remain)/loops, 2)
                
                time_sigmoid_temp.append(end-start)
                
                if loop_remain>0:
                    print('Finnish this run. Time taken: {} seconds. {}% done. {} runs remaining.'.format(round(end-start,3), 
                                                                                                          percentage_done, 
                                                                                                          loop_remain))
                else:
                    time_array = np.array(time_sigmoid_temp)
                    time_total = time_array.sum()
                    print('Time taken for this run: {} seconds. {}% done. Finnished the job in total {} seconds.'.format(round(end-start,3), 
                                                                                                                         percentage_done, 
                                                                                                                         round(time_total,3)))

## Make copies of the accuracy and execution times. Then reverse the temp list.

In [ ]:
accuracy_sigmoid = accuracy_sigmoid_temp.copy()
time_sigmoid = time_sigmoid_temp.copy()

In [ ]:
accuracy_sigmoid_temp.reverse()
time_sigmoid_temp.reverse()

## Iteration loop to create the desired dataframe. Export to csv file.

In [ ]:
temp = []

for d in dropout_prob:
    for l in learning_rate:
        for r in l1_reg:
            for t in train_steps:
                temp.append([d,l,r,t, accuracy_sigmoid_temp.pop(), time_sigmoid_temp.pop()])

df_sigmoid = pd.DataFrame(data=temp, columns=['dropout', 'learning.rate','L1.regularization',
                                      'training.steps','accuracy','execution.time'])

In [ ]:
df_sigmoid.to_csv('Sigmoid_data.csv')
df_sigmoid

## RELU activation

In [ ]:
#Loop counts to keep track of whole training run over the multi-dimensional hyperparams space
loops = len(learning_rate) * len(dropout_prob) * len(train_steps) * len(l1_reg)
loop_remain = loops

#Empty lists to append to
accuracy_relu_temp = []
time_relu_temp = []

#Remove all the unecessary warning
from tensorflow import logging
logging.set_verbosity(logging.ERROR)

for d in dropout_prob:
    for l in learning_rate:
        for r in l1_reg:
            for t in train_steps:
                
                #Start time
                start = time.time()
                
                #Main DNN classifier to train
                classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns, 
                                                        hidden_units=[5,10,5], 
                                                        dropout=d, 
                                                        n_classes=3, 
                                                        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=l, 
                                                                                                    l1_regularization_strength=r), 
                                                        activation_fn=tf.nn.relu)
                
                classifier.train(input_fn=train_input_fn, steps=t)
                
                accuracy_relu_temp.append(classifier.evaluate(input_fn=test_input_fn)['accuracy'])
                
                end = time.time()
                
                loop_remain -= 1
                
                percentage_done = round(100*(loops-loop_remain)/loops, 2)
                
                time_relu_temp.append(end-start)
                
                if loop_remain > 0:
                    print('Finnish this run. Time taken: {} seconds. {}% done. {} runs remaining.'.format(round(end-start,3), 
                                                                                                          percentage_done, 
                                                                                                          loop_remain))
                else:
                    time_array = np.array(time_relu_temp)
                    time_total = time_array.sum()
                    print('Time taken for this run: {} seconds. {}% done. Finnished the job in total {} seconds.'.format(round(end-start,3), 
                                                                                                                         percentage_done, 
                                                                                                                         round(time_total,3)))

In [ ]:
accuracy_relu = accuracy_relu_temp.copy()
time_relu = time_relu_temp.copy()

accuracy_relu_temp.reverse()
time_relu_temp.reverse()

temp = []

for d in dropout_prob:
    for l in learning_rate:
        for r in l1_reg:
            for t in train_steps:
                temp.append([d,l,r,t,accuracy_relu_temp.pop(),time_relu_temp.pop()])

df_relu = pd.DataFrame(data=temp, columns=['dropout', 'learning.rate','L1.regularization',
                                      'training.steps','accuracy','execution.time'])
df_relu.to_csv('RELU_data.csv')
df_relu

## Tanh activation

In [8]:
#Loop counts to keep track of whole training run over the multi-dimensional hyperparams space
loops = len(learning_rate) * len(dropout_prob) * len(train_steps) * len(l1_reg)
loop_remain = loops

#Empty lists to append to
accuracy_tanh_temp = []
time_tanh_temp = []

#Remove all the unecessary warning
from tensorflow import logging
logging.set_verbosity(logging.ERROR)

for d in dropout_prob:
    for l in learning_rate:
        for r in l1_reg:
            for t in train_steps:
                
                start = time.time()
                
                classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns, 
                                                        hidden_units=[5,10,5], 
                                                        dropout=d, 
                                                        n_classes=3, 
                                                        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=l, 
                                                                                                    l1_regularization_strength=r), 
                                                        activation_fn=tf.nn.tanh)
                
                classifier.train(input_fn=train_input_fn, steps=t)
                
                accuracy_tanh_temp.append(classifier.evaluate(input_fn=test_input_fn)['accuracy'])
                
                end = time.time()
                
                loop_remain -= 1
                
                percentage_done = round(100*(loops-loop_remain)/loops, 2)
                
                time_tanh_temp.append(end-start)
                
                if loop_remain > 0:
                    print('Finnish this run. Time taken: {} seconds. {}% done. {} runs remaining.'.format(round(end-start,3), 
                                                                                                          percentage_done, 
                                                                                                          loop_remain))
                else:
                    time_array = np.array(time_tanh_temp)
                    time_total = time_array.sum()
                    print('Time taken for this run: {} seconds. {}% done. Finnished the job in total {} seconds.'.format(round(end-start,3), 
                                                                                                                         percentage_done, 
                                                                                                                         round(time_total,3)))

Finnish this run. Time taken: 5.905 seconds. 0.35% done. 287 runs remaining.
Finnish this run. Time taken: 8.193 seconds. 0.69% done. 286 runs remaining.
Finnish this run. Time taken: 20.765 seconds. 1.04% done. 285 runs remaining.
Finnish this run. Time taken: 5.369 seconds. 1.39% done. 284 runs remaining.
Finnish this run. Time taken: 8.086 seconds. 1.74% done. 283 runs remaining.
Finnish this run. Time taken: 17.353 seconds. 2.08% done. 282 runs remaining.
Finnish this run. Time taken: 5.51 seconds. 2.43% done. 281 runs remaining.
Finnish this run. Time taken: 7.495 seconds. 2.78% done. 280 runs remaining.
Finnish this run. Time taken: 17.509 seconds. 3.12% done. 279 runs remaining.
Finnish this run. Time taken: 5.656 seconds. 3.47% done. 278 runs remaining.
Finnish this run. Time taken: 7.701 seconds. 3.82% done. 277 runs remaining.
Finnish this run. Time taken: 18.205 seconds. 4.17% done. 276 runs remaining.
Finnish this run. Time taken: 5.646 seconds. 4.51% done. 275 runs remaini

NameError: name 'time_relu_temp' is not defined

In [9]:
accuracy_tanh = accuracy_tanh_temp.copy()
time_tanh = time_tanh_temp.copy()

accuracy_tanh_temp.reverse()
time_tanh_temp.reverse()

temp = []

for d in dropout_prob:
    for l in learning_rate:
        for r in l1_reg:
            for t in train_steps:
                temp.append([d,l,r,t,accuracy_tanh_temp.pop(),time_tanh_temp.pop()])

df_tanh = pd.DataFrame(data=temp, columns=['dropout', 'learning.rate','L1.regularization',
                                      'training.steps','accuracy','execution.time'])
df_tanh.to_csv('Tanh_data.csv')
df_tanh

,dropout,learning.rate,L1.regularization,training.steps,accuracy,execution.time
0,0.01,0.0002,0.01,100,0.266667,5.905108
1,0.01,0.0002,0.01,500,0.533333,8.192608
2,0.01,0.0002,0.01,2500,0.533333,20.765186
3,0.01,0.0002,0.10,100,0.500000,5.368610
4,0.01,0.0002,0.10,500,0.433333,8.085875
5,0.01,0.0002,0.10,2500,0.700000,17.352716
6,0.01,0.0002,1.00,100,0.266667,5.510407
7,0.01,0.0002,1.00,500,0.466667,7.494926
8,0.01,0.0002,1.00,2500,0.500000,17.508746
9,0.01,0.0002,10.00,100,0.466667,5.656075


## Exponential linear unit (ELU) activation

In [20]:
#Loop counts to keep track of whole training run over the multi-dimensional hyperparams space
loops = len(learning_rate) * len(dropout_prob) * len(train_steps) * len(l1_reg)
loop_remain = loops

#Empty lists to append to
accuracy_elu_temp = []
time_elu_temp = []

#Remove all the unecessary warning
from tensorflow import logging
logging.set_verbosity(logging.ERROR)

for d in dropout_prob:
    for l in learning_rate:
        for r in l1_reg:
            for t in train_steps:
                
                start = time.time()
                
                classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns, 
                                                        hidden_units=[5,10,5], 
                                                        dropout=d, 
                                                        n_classes=3, 
                                                        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=l, 
                                                                                                    l1_regularization_strength=r), 
                                                        activation_fn=tf.nn.elu)
                
                classifier.train(input_fn=train_input_fn, steps=t)
                
                accuracy_elu_temp.append(classifier.evaluate(input_fn=test_input_fn)['accuracy'])
                
                end = time.time()
                
                loop_remain -= 1
                
                percentage_done = round(100*(loops-loop_remain)/loops, 2)
                
                time_elu_temp.append(end-start)
                
                if loop_remain > 0:
                    print('Finnish this run. Time taken: {} seconds. {}% done. {} runs remaining.'.format(round(end-start,3), 
                                                                                                          percentage_done, 
                                                                                                          loop_remain))
                else:
                    time_array = np.array(time_elu_temp)
                    time_total = time_array.sum()
                    print('Time taken for this run: {} seconds. {}% done. Finnished the job in total {} seconds.'.format(round(end-start,3), 
                                                                                                                         percentage_done, 
                                                                                                                         round(time_total,3)))

Finnish this run. Time taken: 5.497 seconds. 0.35% done. 287 runs remaining.
Finnish this run. Time taken: 7.789 seconds. 0.69% done. 286 runs remaining.
Finnish this run. Time taken: 16.579 seconds. 1.04% done. 285 runs remaining.
Finnish this run. Time taken: 5.716 seconds. 1.39% done. 284 runs remaining.
Finnish this run. Time taken: 7.211 seconds. 1.74% done. 283 runs remaining.
Finnish this run. Time taken: 17.169 seconds. 2.08% done. 282 runs remaining.
Finnish this run. Time taken: 6.085 seconds. 2.43% done. 281 runs remaining.
Finnish this run. Time taken: 8.317 seconds. 2.78% done. 280 runs remaining.
Finnish this run. Time taken: 16.451 seconds. 3.12% done. 279 runs remaining.
Finnish this run. Time taken: 5.839 seconds. 3.47% done. 278 runs remaining.
Finnish this run. Time taken: 7.387 seconds. 3.82% done. 277 runs remaining.
Finnish this run. Time taken: 16.771 seconds. 4.17% done. 276 runs remaining.
Finnish this run. Time taken: 5.458 seconds. 4.51% done. 275 runs remain

In [18]:
accuracy_elu_temp

[0.26666668, 0.26666668]

In [13]:
accuracy_elu = accuracy_elu_temp.copy()
time_elu = time_elu_temp.copy()

accuracy_elu_temp.reverse()
time_elu_temp.reverse()

temp = []

for d in dropout_prob:
    for l in learning_rate:
        for r in l1_reg:
            for t in train_steps:
                temp.append([d,l,r,t,accuracy_elu_temp.pop(),time_elu_temp.pop()])

df_elu = pd.DataFrame(data=temp, columns=['dropout', 'learning.rate','L1.regularization',
                                      'training.steps','accuracy','execution.time'])
df_elu.to_csv('elu_data.csv')
df_elu

IndexError: pop from empty list